In [1]:
import time
import numpy as np
import os
import operator

import keras
from keras import metrics

from keras.models import Model
from keras.layers import Input, RepeatVector, Embedding
from keras.layers.recurrent import LSTM
from keras.utils.training_utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint, TerminateOnNaN

import keras.backend as K
import pickle

from copy import deepcopy
from scipy import spatial

from math import *
import random
from scipy.spatial import distance

Using TensorFlow backend.


In [2]:
model_name = "LSTM_autoencoder"

In [3]:
class ModelMGPU(Model):
    def __init__(self, ser_model, gpus):
        pmodel = multi_gpu_model(ser_model, gpus)
        self.__dict__.update(pmodel.__dict__)
        self._smodel = ser_model

    def __getattribute__(self, attrname):
        '''Override load and save methods to be used from the serial-model. The
        serial-model holds references to the weights in the multi-gpu model.
        '''
        # return Model.__getattribute__(self, attrname)
        if 'load' in attrname or 'save' in attrname:
            return getattr(self._smodel, attrname)

        return super(ModelMGPU, self).__getattribute__(attrname)

In [4]:
class Config() :
    def __init__(self, batch_size, sequence_length, word_embedding_size, lstm_hidden_sizes, epochs) :
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.word_embedding_size = word_embedding_size
        self.lstm_hidden_sizes = lstm_hidden_sizes
        self.epochs = epochs

In [5]:
class Autoencoder() :
    def __init__(self, embedding_weight_path, word_dic_path, model_path, config) :
        try :
            assert os.path.exists(embedding_weight_path)
            self.embedding_weight_path = embedding_weight_path     
            self.embedding_weight = np.load(self.embedding_weight_path)
            print("self.embedding_weight.shape : ", self.embedding_weight)
            
        except AssertionError :
            print(embedding_weight_path, " >>>>>> 존재하지 않습니다.")
            quit()
            
        try :
            assert os.path.exists(word_dic_path)
            self.word_dic_path = word_dic_path
            self.word2id, self.id2word = pickle.load(open(self.word_dic_path, "rb"), encoding = 'utf-8')
            self.vocab_size = len(self.word2id)
            print("self.vocab_size : ", self.vocab_size)
            
        except AssertionError :
            print(word_dic_path, " >>>>>> 존재하지 않습니다.")
            quit()
            
        self.model_path = model_path
        if (not os.path.exists(self.model_path)) :
            os.makedirs(self.model_path)
            
        self.pictures_path = self.model_path + "/picutres"
        self.weights_path = self.model_path + "/weights"
        self.architectures_path = self.model_path + "/architecture"
        
        if (not os.path.exists(self.pictures_path)) :
            os.makedirs(self.pictures_path)
        if (not os.path.exists(self.weights_path)) :
            os.makedirs(self.weights_path)
        if (not os.path.exists(self.architectures_path)) :
            os.makedirs(self.architectures_path)
        
        self.batch_size = config.batch_size
        self.sequence_length = config.sequence_length
        self.word_embedding_size = config.word_embedding_size
        self.lstm_hidden_sizes = config.lstm_hidden_sizes
        self.epochs = config.epochs
    
    def make_autoencoder(self) :
        original_sequences = Input(shape = (self.sequence_length, ), name = 'original_sequence')
        embedding_sequences = Embedding(self.embedding_weight.shape[0], self.embedding_weight.shape[1],
                                       weights = [self.embedding_weight], input_length = self.sequence_length,
                                       trainable = False, name = 'embedding_sequence')(original_sequences)
        self.embedding_model = Model(inputs = original_sequences, outputs = embedding_sequences)
        print("self.embedding_model:\n{}\n".format(self.embedding_model.summary()))

        ae_input = Input(shape = (self.sequence_length, self.word_embedding_size, ), name = 'ae_input')
        hidden_space = self.encoder_model(ae_input)        
        reconstructed_sequences = self.decoder_model(hidden_space)
        
        self.autoencoder = Model(inputs = ae_input, outputs = reconstructed_sequences)
        self.parallel_ae = ModelMGPU(self.autoencoder, gpus = 2)
        self.parallel_ae.compile(optimizer = 'Adam', loss = 'mse', metrics = [metrics.mse])
        print("self.autoencoder:\n{}\n".format(self.autoencoder.summary()))

        with open(self.architectures_path + "/autoencoder.json", "w", encoding = 'utf-8') as fp :
            fp.write((self.autoencoder).to_json())            

    def encoder_model(self, encoder_input) :
        # [batch_size, sequence_length, 256]
        encoder_layer1 = LSTM(self.lstm_hidden_sizes[0], return_sequences = True, name = 'encoder_layer1')(encoder_input)
        
        # [batch_size, sequence_length, 128]
        hidden_info = LSTM(self.lstm_hidden_sizes[1], return_sequences = False, name = 'encoder_layer2')(encoder_layer1)

        return hidden_info
        
    def decoder_model(self, decoder_input) :       
        # [batch_size, sequence_length, 128]
        repeater = RepeatVector(self.sequence_length, name = 'Repeater')(decoder_input)
        
        # [batch_size, sequence_length, 256]
        decoder_layer1 = LSTM(self.lstm_hidden_sizes[0], return_sequences = True, name = 'decoder_layer1')(repeater)

        # [batch_size, sequence_length, 512]
        reconstructed_sequences = LSTM(self.word_embedding_size, return_sequences = True,
                                      name = 'reconstructed_sequences')(decoder_layer1)
        
        return reconstructed_sequences           
    
    # data를 Id로 바꿔줌
    def convertToId(self, sequences) :
        converted_data = []
        for sequence in sequences :
            splitted_sequences = sequence.split()
            wordList = []
            
            for word in splitted_sequences :
                if (word.endswith("/noun") or word.endswith("/verb") or word.endswith("adjective") or 
                    word.endswith('/alpha') or word.endswith("/number")) :
                    if word not in (self.word2id).keys() :
                        pass
                    else :
                        wordList.append(self.word2id[word])

                if (len(wordList) >= self.sequence_length) :
                    break
                    
            converted_data.append(deepcopy(wordList))
            del wordList
        
        final_input = keras.preprocessing.sequence.pad_sequences(converted_data,
                                                                maxlen = self.sequence_length,
                                                                dtype = 'int32',
                                                                padding = 'pre',
                                                                truncating = 'post',
                                                                value = self.word2id["_PAD"])
        
        return final_input
    
    def load_weights(self, ae_weights_path) :
        print("{} <<<< 저장된 weight를 불러옵니다.\n".format(ae_weights_path))
        (self.parallel_ae).load_weights(ae_weights_path)
                
    # 이미 훈련한 모델을 재사용할 경우 load = True로 호출
    def train(self, train_data_path, valid_data_path, load = False, ae_weights_path = 0) :             
        if load : self.load_weights(ae_weights_path)
        else : print("저장된 weights를 불러오지 않습니다.\n")
            
        self.encoder = Model(self.parallel_ae.input,
                            self.parallel_ae.layers[3].layers[2].output)
        print("self.encoder:\n{}\n".format(self.encoder.summary()))        

        train_sentences = []
        valid_sentences = []
        with open(train_data_path, "r", encoding = 'utf-8') as fp :
            temp = fp.readlines()
            for sentence in temp :
                split_ = sentence.strip().split("\t")
                train_sentences.append(split_[3])

        
        with open(valid_data_path, "r", encoding = 'utf-8') as fp :
            temp = fp.readlines()
            for sentence in temp :
                split_ = sentence.strip().split("\t")
                valid_sentences.append(split_[3])

        total_data_length = len(train_sentences)
        iters = ceil(total_data_length / self.batch_size)
        print("total_data_length : {:10d} // iters : {:5d}".format(total_data_length, iters))
        valid_iters = ceil(len(valid_sentences) / (self.batch_size * 2))
        print("valid_iters : ", valid_iters)
        
        train_batch = []
        valid_batch = []
        
        train_input = self.convertToId(train_sentences)
        valid_input = self.convertToId(valid_sentences)
                
        for i in range(self.epochs) :
            sum_ae_loss = 0
            train_total = 0
            print("\nepoch : ", i + 1)
            
            for iter_ in range(iters) :
                # training
                if (iter_ == iters - 1) : # 마지막
                    train_batch = train_input[self.batch_size * iter_ : ]
                    
                else :
                    train_batch = train_input[self.batch_size * iter_ : self.batch_size * (iter_ + 1)]
    
                embedding_output = self.embedding_model.predict_on_batch(x = train_batch)
                
                if (iter_ == 0) :
                    print("embedding_output.shape:\n{}\n".format(embedding_output.shape))
                
                ae_loss = self.parallel_ae.train_on_batch(embedding_output, embedding_output)
                sum_ae_loss = ae_loss[1]

                print("iter_ [{:5d} / {:5d}] ----> mse [{:8.6f}]".format(iter_, iters, sum_ae_loss))
                
            print("\nautoencoder 평가")    
            cos_sim = [0, ] * 10
            sim_per = 0
            valid_total = 0
            for valid_iter in range(valid_iters) :
                if (valid_iter == valid_iters - 1) : # 마지막
                    valid_batch = valid_input[self.batch_size * 2 * valid_iter : ]
                    
                else :
                    valid_batch = valid_input[self.batch_size * 2 * valid_iter : self.batch_size * 2 * (valid_iter + 1)]
                valid_total += len(valid_batch)
                        
                # sequence_input을 embedding시킴
                embedding_output = self.embedding_model.predict_on_batch(valid_batch)
                        
                # embedding_output을 parallel_ae에 넣어, reconstructed input을 얻음
                rec_valid = self.parallel_ae.predict_on_batch(embedding_output)
                        
                # original input의 encoder ouput을 얻음
                Xvec = self.encoder.predict_on_batch(embedding_output)
                        
                # reconstructed input의 enocder output을 얻음
                Yvec = self.encoder.predict_on_batch(rec_valid)
                                
                # 얻은 두 개의 encoder output의 유사도(cosine similarity)를 확인
                for j in range(Xvec.shape[0]) :
                    result = 1 - spatial.distance.cosine(Xvec[j], Yvec[j])
                    idx_ = int(result * 10)
                        
                    if (idx_ == 10) :
                        cos_sim[9] += 1
                    else :
                        cos_sim[idx_] += 1

                sim_per = [round(cos_sim[j] / float(valid_total), 5) for j in range(0, 10)]
                print("valid_iter [{:2d} / {:2d}] ---- percentage : {}".format(valid_iter, valid_iters, sim_per))

            self.parallel_ae.save_weights(self.weights_path + '/epoch_{:d}'.format(i + 1) + '_iter:{:d}'.format(iter_) +
                            '_loss{:.4f}'.format(sum_ae_loss) + '_sim{:.4f}'.format(sim_per[-1]) + ".hdf5") 
            
    def test(self, ae_weight_path, test_data_path) :
        test_sentences = []
        with open(test_data_path, "r", encoding = 'utf-8') as fp :
            temp = fp.readlines()
            for sentence in temp :
                test_sentences.append(sentence.split("\t")[1])

        self.load_weights(ae_weight_path)
        self.encoder = Model(self.parallel_ae.input,
                            self.parallel_ae.layers[3].layers[2].output)
        print("self.encoder:\n{}\n".format(self.encoder.summary()))

        test_input = self.convertToId(test_sentences)
        
        total_test_length = len(test_sentences)
        test_iters = ceil(total_test_length / float(self.batch_size * 4))
        print("total_test_length : [{:6d}] -- iters : [{:3d}]".format(total_test_length, test_iters))
        
        cos_sim = [0, ] * 10
        sim_per = 0
        test_total = 0
        test_batch = []
        
        for test_iter in range(test_iters) :
            if (test_iter == test_iters - 1) : # 마지막
                test_batch = test_input[self.batch_size * 4 * test_iter : ]
                    
            else :
                test_batch = test_input[self.batch_size * 4 * test_iter : self.batch_size * 4 * (test_iter + 1)]
            test_total += len(test_batch)
                        
            # sequence_input을 embedding시킴
            embedding_output = self.embedding_model.predict_on_batch(test_batch)
                        
            # embedding_output을 parallel_ae에 넣어, reconstructed input을 얻음
            rec_valid = self.parallel_ae.predict_on_batch(embedding_output)
                        
            # original input의 encoder ouput을 얻음
            Xvec = self.encoder.predict_on_batch(embedding_output)
                        
            # reconstructed input의 enocder output을 얻음
            Yvec = self.encoder.predict_on_batch(rec_valid)
                                
            # 얻은 두 개의 encoder output의 유사도(cosine similarity)를 확인
            for j in range(Xvec.shape[0]) :
                result = 1 - spatial.distance.cosine(Xvec[j], Yvec[j])
                idx_ = int(result * 10)
                        
                if (idx_ == 10) :
                    cos_sim[9] += 1
                else :
                    cos_sim[idx_] += 1
                    
            sim_per = [round(cos_sim[j] / float(test_total), 5) for j in range(0, 10)]
            print("valid_iter [{:2d} / {:2d}] ---- percentage : {}".format(test_iter, test_iters, sim_per))    

In [6]:
batch_size = 2048
sequence_length = 300
word_embedding_size = 512
lstm_hidden_sizes = [256, 128]
epochs = 100
config = Config(batch_size, sequence_length, word_embedding_size, lstm_hidden_sizes, epochs)

In [7]:
embedding_weight_path = "./data_20181011/trainingSet/word_embedding_weight.npy"
word_dic_path = "./data_20181011/trainingSet/word2id_and_id2word.pkl"
model_path = model_name
AE = Autoencoder(embedding_weight_path, word_dic_path, model_path, config)

self.embedding_weight.shape :  [[ 0.02557867 -0.09405205  0.09645993 ...  0.07120728  0.10891931
   0.04693465]
 [ 0.24249735 -0.18350758 -0.02219512 ...  0.1278952   0.00920443
   0.048867  ]
 [ 0.02898274 -0.20504563  0.01854682 ...  0.03058945  0.0938068
  -0.15830368]
 ...
 [-0.07557604  0.01604056  0.51072717 ...  0.16818266  0.68855447
   0.12199204]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.1         0.1         0.1        ...  0.1         0.1
   0.1       ]]
self.vocab_size :  152683


In [8]:
AE.make_autoencoder()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
original_sequence (InputLaye (None, 300)               0         
_________________________________________________________________
embedding_sequence (Embeddin (None, 300, 512)          78173696  
Total params: 78,173,696
Trainable params: 0
Non-trainable params: 78,173,696
_________________________________________________________________
self.embedding_model:
None

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ae_input (InputLayer)        (None, 300, 512)          0         
_________________________________________________________________
encoder_layer1 (LSTM)        (None, 300, 256)          787456    
_________________________________________________________________
encoder_layer2 (LSTM)        (None, 128)               197120    
____________________________________

In [9]:
train_data_path = "./data_20181011/trainingSet/excluded_total_dataset.txt"
valid_data_path = "./data_20181011/testingSet/merged_preprocessedNaverNews.txt"
AE.train(train_data_path, valid_data_path)

저장된 weights를 불러오지 않습니다.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ae_input (InputLayer)        (None, 300, 512)          0         
_________________________________________________________________
encoder_layer1 (LSTM)        (None, 300, 256)          787456    
_________________________________________________________________
encoder_layer2 (LSTM)        (None, 128)               197120    
Total params: 984,576
Trainable params: 984,576
Non-trainable params: 0
_________________________________________________________________
self.encoder:
None

autoencoder metrics ======>  ['loss', 'mean_squared_error']
train_sentences 저장
train_sentences[-1] : 
미국/noun 이란/noun 정면/noun 충돌/noun 시나리오/noun 대로/josa 가나/noun 내/determiner 일/modifier 신문/noun 300/number 개/noun 공격/noun 목표/noun 세/noun 방향/noun 에서/josa 타격/noun 계획/noun 미국/noun 의/josa 강하다/adjective 부인/noun 에도/josa 불구/noun 하고/josa 이란/noun 공격/noun 시나리오/noun 가/j

self.convertToId(valid_sentences)

epoch :  1
embedding_output.shape:
(2048, 300, 512)

iter_ [    0 /   390] ----> mse [0.033100]
iter_ [    1 /   390] ----> mse [0.032649]
iter_ [    2 /   390] ----> mse [0.032344]
iter_ [    3 /   390] ----> mse [0.031574]
iter_ [    4 /   390] ----> mse [0.031707]
iter_ [    5 /   390] ----> mse [0.030693]
iter_ [    6 /   390] ----> mse [0.030759]
iter_ [    7 /   390] ----> mse [0.030634]
iter_ [    8 /   390] ----> mse [0.030516]
iter_ [    9 /   390] ----> mse [0.030698]
iter_ [   10 /   390] ----> mse [0.030141]
iter_ [   11 /   390] ----> mse [0.030300]
iter_ [   12 /   390] ----> mse [0.030698]
iter_ [   13 /   390] ----> mse [0.030329]
iter_ [   14 /   390] ----> mse [0.030213]
iter_ [   15 /   390] ----> mse [0.030122]
iter_ [   16 /   390] ----> mse [0.030309]
iter_ [   17 /   390] ----> mse [0.030062]
iter_ [   18 /   390] ----> mse [0.029859]
iter_ [   19 /   390] ----> mse [0.030199]
iter_ [   20 /   390] ----> mse [0.030071]
iter_ [  

iter_ [  189 /   390] ----> mse [0.029801]
iter_ [  190 /   390] ----> mse [0.030269]
iter_ [  191 /   390] ----> mse [0.030265]
iter_ [  192 /   390] ----> mse [0.030792]
iter_ [  193 /   390] ----> mse [0.031611]
iter_ [  194 /   390] ----> mse [0.031823]
iter_ [  195 /   390] ----> mse [0.030808]
iter_ [  196 /   390] ----> mse [0.027339]
iter_ [  197 /   390] ----> mse [0.027191]
iter_ [  198 /   390] ----> mse [0.027048]
iter_ [  199 /   390] ----> mse [0.027530]
iter_ [  200 /   390] ----> mse [0.027601]
iter_ [  201 /   390] ----> mse [0.026927]
iter_ [  202 /   390] ----> mse [0.027168]
iter_ [  203 /   390] ----> mse [0.026754]
iter_ [  204 /   390] ----> mse [0.026893]
iter_ [  205 /   390] ----> mse [0.027317]
iter_ [  206 /   390] ----> mse [0.027114]
iter_ [  207 /   390] ----> mse [0.027297]
iter_ [  208 /   390] ----> mse [0.027088]
iter_ [  209 /   390] ----> mse [0.027190]
iter_ [  210 /   390] ----> mse [0.026884]
iter_ [  211 /   390] ----> mse [0.027146]
iter_ [  21

iter_ [  380 /   390] ----> mse [0.027123]
iter_ [  381 /   390] ----> mse [0.026773]
iter_ [  382 /   390] ----> mse [0.026763]
iter_ [  383 /   390] ----> mse [0.027231]
iter_ [  384 /   390] ----> mse [0.026651]
iter_ [  385 /   390] ----> mse [0.026898]
iter_ [  386 /   390] ----> mse [0.026953]
iter_ [  387 /   390] ----> mse [0.027034]
iter_ [  388 /   390] ----> mse [0.026941]
iter_ [  389 /   390] ----> mse [0.027415]

autoencoder 평가
valid_iter [ 0 /  8] ---- percentage : [0.00635, 0.00391, 0.00708, 0.00708, 0.01123, 0.00757, 0.00854, 0.05396, 0.08081, 0.81348]
valid_iter [ 1 /  8] ---- percentage : [0.06287, 0.01892, 0.01111, 0.00916, 0.0116, 0.00757, 0.01025, 0.06775, 0.09009, 0.71069]
valid_iter [ 2 /  8] ---- percentage : [0.06348, 0.01782, 0.01302, 0.01107, 0.01204, 0.00895, 0.01131, 0.06828, 0.08993, 0.7041]
valid_iter [ 3 /  8] ---- percentage : [0.05005, 0.0152, 0.01282, 0.01068, 0.01282, 0.00934, 0.01147, 0.06207, 0.0871, 0.72845]
valid_iter [ 4 /  8] ---- percentage :

iter_ [  155 /   390] ----> mse [0.030285]
iter_ [  156 /   390] ----> mse [0.030243]
iter_ [  157 /   390] ----> mse [0.030120]
iter_ [  158 /   390] ----> mse [0.029985]
iter_ [  159 /   390] ----> mse [0.030021]
iter_ [  160 /   390] ----> mse [0.030401]
iter_ [  161 /   390] ----> mse [0.029975]
iter_ [  162 /   390] ----> mse [0.030107]
iter_ [  163 /   390] ----> mse [0.030107]
iter_ [  164 /   390] ----> mse [0.030067]
iter_ [  165 /   390] ----> mse [0.029778]
iter_ [  166 /   390] ----> mse [0.030190]
iter_ [  167 /   390] ----> mse [0.029847]
iter_ [  168 /   390] ----> mse [0.030085]
iter_ [  169 /   390] ----> mse [0.030095]
iter_ [  170 /   390] ----> mse [0.030142]
iter_ [  171 /   390] ----> mse [0.030261]
iter_ [  172 /   390] ----> mse [0.030012]
iter_ [  173 /   390] ----> mse [0.030135]
iter_ [  174 /   390] ----> mse [0.029940]
iter_ [  175 /   390] ----> mse [0.029603]
iter_ [  176 /   390] ----> mse [0.030087]
iter_ [  177 /   390] ----> mse [0.030145]
iter_ [  17

...

iter_ [  272 /   390] ----> mse [0.028534]
iter_ [  273 /   390] ----> mse [0.028499]
iter_ [  274 /   390] ----> mse [0.028528]
iter_ [  275 /   390] ----> mse [0.028404]
iter_ [  276 /   390] ----> mse [0.028181]
iter_ [  277 /   390] ----> mse [0.028506]
iter_ [  278 /   390] ----> mse [0.028259]
iter_ [  279 /   390] ----> mse [0.028091]
iter_ [  280 /   390] ----> mse [0.028446]
iter_ [  281 /   390] ----> mse [0.028458]
iter_ [  282 /   390] ----> mse [0.028259]
iter_ [  283 /   390] ----> mse [0.028593]
iter_ [  284 /   390] ----> mse [0.028547]
iter_ [  285 /   390] ----> mse [0.028449]
iter_ [  286 /   390] ----> mse [0.028703]
iter_ [  287 /   390] ----> mse [0.028420]
iter_ [  288 /   390] ----> mse [0.028261]
iter_ [  289 /   390] ----> mse [0.028296]
iter_ [  290 /   390] ----> mse [0.028330]
iter_ [  291 /   390] ----> mse [0.028471]
iter_ [  292 /   390] ----> mse [0.028296]
iter_ [  293 /   390] ----> mse [0.028249]
iter_ [  294 /   390] ----> mse [0.028173]
iter_ [  29

iter_ [   52 /   390] ----> mse [0.029243]
iter_ [   53 /   390] ----> mse [0.029205]
iter_ [   54 /   390] ----> mse [0.029113]
iter_ [   55 /   390] ----> mse [0.029013]
iter_ [   56 /   390] ----> mse [0.029476]
iter_ [   57 /   390] ----> mse [0.028982]
iter_ [   58 /   390] ----> mse [0.029164]
iter_ [   59 /   390] ----> mse [0.028786]
iter_ [   60 /   390] ----> mse [0.028896]
iter_ [   61 /   390] ----> mse [0.029020]
iter_ [   62 /   390] ----> mse [0.029060]
iter_ [   63 /   390] ----> mse [0.029164]
iter_ [   64 /   390] ----> mse [0.028879]
iter_ [   65 /   390] ----> mse [0.029023]
iter_ [   66 /   390] ----> mse [0.026700]
iter_ [   67 /   390] ----> mse [0.026396]
iter_ [   68 /   390] ----> mse [0.026560]
iter_ [   69 /   390] ----> mse [0.026620]
iter_ [   70 /   390] ----> mse [0.026500]
iter_ [   71 /   390] ----> mse [0.026430]
iter_ [   72 /   390] ----> mse [0.026443]
iter_ [   73 /   390] ----> mse [0.026609]
iter_ [   74 /   390] ----> mse [0.026432]
iter_ [   7

iter_ [  243 /   390] ----> mse [0.025719]
iter_ [  244 /   390] ----> mse [0.025685]
iter_ [  245 /   390] ----> mse [0.025671]
iter_ [  246 /   390] ----> mse [0.025793]
iter_ [  247 /   390] ----> mse [0.026007]
iter_ [  248 /   390] ----> mse [0.025774]
iter_ [  249 /   390] ----> mse [0.025717]
iter_ [  250 /   390] ----> mse [0.025767]
iter_ [  251 /   390] ----> mse [0.025778]
iter_ [  252 /   390] ----> mse [0.025998]
iter_ [  253 /   390] ----> mse [0.025684]
iter_ [  254 /   390] ----> mse [0.026069]
iter_ [  255 /   390] ----> mse [0.025732]
iter_ [  256 /   390] ----> mse [0.025516]
iter_ [  257 /   390] ----> mse [0.026208]
iter_ [  258 /   390] ----> mse [0.025942]
iter_ [  259 /   390] ----> mse [0.025845]
iter_ [  260 /   390] ----> mse [0.025847]
iter_ [  261 /   390] ----> mse [0.027249]
iter_ [  262 /   390] ----> mse [0.028151]
iter_ [  263 /   390] ----> mse [0.028375]
iter_ [  264 /   390] ----> mse [0.028374]
iter_ [  265 /   390] ----> mse [0.028388]
iter_ [  26

iter_ [   23 /   390] ----> mse [0.029095]
iter_ [   24 /   390] ----> mse [0.029066]
iter_ [   25 /   390] ----> mse [0.029047]
iter_ [   26 /   390] ----> mse [0.029145]
iter_ [   27 /   390] ----> mse [0.028934]
iter_ [   28 /   390] ----> mse [0.029056]
iter_ [   29 /   390] ----> mse [0.028981]
iter_ [   30 /   390] ----> mse [0.029095]
iter_ [   31 /   390] ----> mse [0.028913]
iter_ [   32 /   390] ----> mse [0.028871]
iter_ [   33 /   390] ----> mse [0.029485]
iter_ [   34 /   390] ----> mse [0.029110]
iter_ [   35 /   390] ----> mse [0.028883]
iter_ [   36 /   390] ----> mse [0.029136]
iter_ [   37 /   390] ----> mse [0.028727]
iter_ [   38 /   390] ----> mse [0.029144]
iter_ [   39 /   390] ----> mse [0.029355]
iter_ [   40 /   390] ----> mse [0.029083]
iter_ [   41 /   390] ----> mse [0.029073]
iter_ [   42 /   390] ----> mse [0.029177]
iter_ [   43 /   390] ----> mse [0.028995]
iter_ [   44 /   390] ----> mse [0.028794]
iter_ [   45 /   390] ----> mse [0.029141]
iter_ [   4

KeyboardInterrupt: 